In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
import json
%matplotlib inline

In [2]:
candidate = pd.read_excel("candidate_master_data.xlsx",sheet_name='Candidate Data')
candidate.to_csv("candidate_master.csv",index=None,header=True)
cand_d = pd.read_csv("candidate_master.csv")

In [3]:
def skill_parsing(skill):
    '''
    Function that extracts the skills from the Skill column and creates seperate primary and secondary skillsets with experience
    in each skill. This will be later appended to the main table dropping the main Skill column.
    '''
    res = json.loads(skill,strict=False)
    skill_list=res['skills']
    
    primary_skill = {}
    secondary_skill = {}

    for i in range (len(skill_list)):
        if skill_list[i]['skill_type'] == 'primary':
            primary_skill[skill_list[i]['skill']]=skill_list[i]['total_exp_in_each_skill']
        else:
            if skill_list[i]['skill_type'] == 'secondary':
                secondary_skill[skill_list[i]['skill']]=skill_list[i]['total_exp_in_each_skill']
    return primary_skill, secondary_skill

In [4]:
import tensorflow as tf
import numpy as np
import more_itertools
import json
from neo4j import GraphDatabase, Driver

class GraphSequence(tf.keras.utils.Sequence):

	def __init__(self, args, batch_size=32, test=False):
		self.batch_size = batch_size
		
		self.query = """
			MATCH 
            
			RETURN as x, score as y
		"""

		self.query_params = {
			"dataset_name": "article_0",
			"test": test
		}

		with open('./settings.json') as f:
			self.settings = json.load(f)[args.database]

		driver = GraphDatabase.driver(
			self.settings["neo4j_url"], 
			auth=(self.settings["neo4j_user"], self.settings["neo4j_password"]))

		with driver.session() as session:
			data = session.run(self.query, **self.query_params).data()
			data = [ (np.array(i["x"]), i["y"]) for i in data]
			
			# Split the data up into "batches"
			data = more_itertools.chunked(data, self.batch_size)

			# Format our batches in the way Keras expects them:
			# An array of tuples (x_batch, y_batch)

			# An x_batch is a numpy array of shape (batch_size, 12), 
			# containing the concatenated style and style_preference vectors. 

			# A y_batch is a numpy array of shape (batch_size,1) containing the review scores.

			self.data = [ (np.array([j[0] for j in i]), np.array([j[1] for j in i])) for i in data]


	def __len__(self):
		return len(self.data)

	def __getitem__(self, idx):
		return self.data[idx]